In [105]:
from google.colab import drive
from sklearn.inspection import permutation_importance
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from pandas.core.common import random_state
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import joblib

In [106]:
drive.mount('/content/drive')
%cd drive/MyDrive/ML_final
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/ML_final'
/content/drive/MyDrive/ML_final


In [107]:
data = pd.concat([train, test])
leave_col = [f for f in data.columns if f.startswith('measurement')
             or f == 'id' or f == 'product_code' or f == 'loading' or f
             == 'failure']
data = data[leave_col]
data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
pro_code = [
    'A',
    'B',
    'C',
    'D',
    'E',
    'F',
    'G',
    'H',
    'I',
    ]
fill_col = [f for f in data.columns if f.startswith('measurement') or f
            == 'loading']
for code in pro_code:
    tem_data = data[data.product_code == code].drop(['failure',
            'product_code', 'id', 'm3_missing', 'm5_missing'], axis=1)
    model_knn = KNNImputer(n_neighbors=5)
    tem_data = model_knn.fit_transform(tem_data)
    data.loc[data.product_code == code, fill_col] = tem_data
data_scal = data[fill_col]
scaler = StandardScaler().fit(data_scal)
data_scal = scaler.transform(data_scal)
data[fill_col] = data_scal
test_data = data[data.failure.isnull()]
test_x = test_data.drop(['failure', 'product_code', 'id'], axis=1)
select_f = [
    'measurement_2',
    'loading',
    'measurement_17',
    'measurement_1',
    'm3_missing',
    'm5_missing'
    ]
test_x_s = test_x[select_f]

In [108]:
test_x_s=test_x_s.rename(columns = {'measurement_6':'measurement_1'})

In [109]:
model = joblib.load('final_model')
pred = model.predict_proba(test_x_s)
pred = pred[:, 1]
id_list = []
pred_list = []
for i in test_data['id']:
    id_list.append(i)
for i in pred:
    pred_list.append(i)
output_data = {'id': id_list, 'failure': pred_list}
output = pd.DataFrame(output_data)
output.to_csv('output.csv', encoding='utf-8', index=False)